# DialogHelper Integration Tests

This notebook tests the dialoghelper integration with Dialeng, including:
- Basic cell operations (read_msg, find_msgs, add_msg)
- Cell content editing (msg_str_replace, msg_insert_line)
- Cell properties (update_msg, pinned, skipped)
- JavaScript injection via iife()
- HTML injection via add_html()
- Bidirectional data transfer (push_data/pop_data)
- Cell execution via run_msg()

**Run each cell in order to test the features.**

In [ ]:
# Cell 1: Setup & Import
# Configure dialoghelper to connect to our server
from dialoghelper import dh_settings
dh_settings['port'] = 8000  # Dialeng server port

from dialoghelper import (
    read_msg, find_msgs, update_msg, add_msg, del_msg,
    msg_str_replace, msg_insert_line, msg_strs_replace, msg_replace_lines,
    add_html, iife, fire_event, event_get, run_msg,
    curr_dialog, msg_idx
)
print("DialogHelper configured!")
print(f"Port: {dh_settings['port']}")

In [ ]:
# Cell 2: Test curr_dialog - Get Dialog/Notebook Info
# Note: curr_dialog() returns a plain dict, not an object
dialog = curr_dialog()
print(f"Dialog name: {dialog['name']}")
print(f"Dialog mode: {dialog['mode']}")

In [ ]:
# Cell 3: Test read_msg - Read Cell Content
# Note: read_msg() returns {'msg': {...}} - access via .msg
prev = read_msg(-1)
print(f"Previous cell type: {prev.msg.type}")
print(f"Previous cell content preview: {prev.msg.content[:50]}...")

# Read current cell (n=0 means current)
current = read_msg(0)
print(f"\nCurrent cell ID: {current.msg.id}")
print(f"Current cell type: {current.msg.type}")

In [ ]:
# Cell 4: Test msg_idx - Get Cell Index
# Get the index of the setup cell
setup_idx = msg_idx(msgid='setup')
print(f"Setup cell is at index: {setup_idx}")

# Get current cell index (access ID via .msg.id)
current = read_msg(0)
current_idx = msg_idx(msgid=current.msg.id)
print(f"Current cell is at index: {current_idx}")

In [ ]:
# Cell 5: Test find_msgs - Search Cells
# Note: find_msgs returns list of dicts with 'content' field (not 'source')
code_cells = find_msgs(msg_type="code", limit=10)
print(f"Found {len(code_cells)} code cells:")
for cell in code_cells[:5]:
    print(f"  - {cell.id}: {cell.content[:40]}...")

# Find cells matching a regex pattern
test_cells = find_msgs(re_pattern="Test", limit=5)
print(f"\nFound {len(test_cells)} cells with 'Test' in content")

In [ ]:
# Cell 6: Test update_msg - Update Cell Properties
# Get current cell (access via .msg)
current = read_msg(0)
print(f"Current cell ID: {current.msg.id}")

# Pin the cell (keeps it in LLM context)
update_msg(msgid=current.msg.id, pinned=True)
print("Cell pinned!")

# Read back to verify (access pinned via .msg.pinned)
updated = read_msg(0)
print(f"Pinned status: {updated.msg.pinned}")

# Unpin it
update_msg(msgid=current.msg.id, pinned=False)
print("Cell unpinned!")

In [ ]:
# Cell 7: Test add_msg - Create New Cells
# Note: add_msg returns just the cell ID string, not an object

# Add a note cell after this one
note_cell_id = add_msg(
    "## Auto-generated Note\nThis note was created by Python code!",
    msg_type="note",
    placement="after"
)
print(f"Created note cell: {note_cell_id}")

# Add a code cell
code_cell_id = add_msg(
    "# Auto-generated code cell\nprint('Hello from generated cell!')",
    msg_type="code",
    placement="after"
)
print(f"Created code cell: {code_cell_id}")

In [ ]:
# Cell 8: Test msg_str_replace - Replace String in Cell
# First, create a cell to modify
test_cell_id = add_msg(
    "Hello World! This is a TEST message.",
    msg_type="note",
    placement="after"
)
print(f"Created cell: {test_cell_id}")

# Read the cell to see original content (access via .msg.content)
test_cell = read_msg(msgid=test_cell_id)
print(f"Original: {test_cell.msg.content}")

# Replace a string
msg_str_replace(msgid=test_cell_id, old_str="World", new_str="Dialeng")

# Read back to verify
updated = read_msg(msgid=test_cell_id)
print(f"After replace: {updated.msg.content}")

In [ ]:
# Cell 9: Test msg_insert_line - Insert Line in Cell
# Create a multi-line cell
test_cell_id = add_msg(
    "Line 1\nLine 2\nLine 3",
    msg_type="note",
    placement="after"
)
print(f"Created cell: {test_cell_id}")

# Read the cell to see original content (access via .msg.content)
test_cell = read_msg(msgid=test_cell_id)
print(f"Original:\n{test_cell.msg.content}")

# Insert a line after line 1 (0-indexed)
msg_insert_line(msgid=test_cell_id, insert_line=1, new_str="Inserted Line!")

# Read back to verify
updated = read_msg(msgid=test_cell_id)
print(f"\nAfter insert:\n{updated.msg.content}")

In [ ]:
# Cell 10: Test del_msg - Delete Cell
# Create a temporary cell
temp_cell_id = add_msg(
    "This cell will be deleted",
    msg_type="note",
    placement="after"
)
print(f"Created cell: {temp_cell_id}")

# Count cells before
cells_before = find_msgs(limit=100)
print(f"Cells before deletion: {len(cells_before)}")

# Delete the cell
del_msg(msgid=temp_cell_id)
print("Cell deleted!")

# Count cells after
cells_after = find_msgs(limit=100)
print(f"Cells after deletion: {len(cells_after)}")

## JavaScript Injection Tests (iife)

The following cells test `iife()` which allows executing JavaScript in the browser from Python.

In [ ]:
# Cell 11: Test iife - Console Log
# Simple console.log to verify JS injection works
iife("console.log('Hello from Python via iife!')")
print("Check browser console (F12 > Console) for the message!")

In [ ]:
# Cell 12: Test iife - Alert Box
# Simple alert to verify JS injection works
iife("alert('Hello from Python via iife!')")
print("Alert triggered! Check for popup.")

In [ ]:
# Cell 13: Test iife - DOM Manipulation
# Change page background color temporarily
iife("""
    const original = document.body.style.backgroundColor;
    document.body.style.backgroundColor = '#e8f5e9';  // Light green
    setTimeout(() => {
        document.body.style.backgroundColor = original;
    }, 2000);
    console.log('Background changed for 2 seconds');
""")
print("Check the page - background should flash green!")

In [ ]:
# Cell 14: Test iife - Add Custom UI Element
# Inject a floating notification
iife("""
    const notif = document.createElement('div');
    notif.id = 'test-notification';
    notif.style.cssText = `
        position: fixed;
        top: 20px;
        right: 20px;
        padding: 15px 25px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 8px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.2);
        z-index: 10000;
        font-family: system-ui;
    `;
    notif.textContent = 'Injected from Python!';
    document.body.appendChild(notif);
    setTimeout(() => notif.remove(), 3000);
""")
print("Look at the top-right corner!")

In [ ]:
# Cell 15: Test add_html - Direct HTML Injection
# Inject HTML with HTMX OOB swap
add_html('''
<div hx-swap-oob="beforeend:#js-script">
    <script>
        console.log('Script injected via add_html!');
        const banner = document.createElement('div');
        banner.style.cssText = 'padding: 10px; margin: 10px 0; background: #d4edda; border-radius: 4px; border: 1px solid #c3e6cb;';
        banner.textContent = 'This banner was injected via add_html()!';
        document.getElementById('js-script').appendChild(banner);
    </script>
</div>
''')
print("HTML/Script injected! Check for green banner at bottom of page.")

## Bidirectional Data Transfer Tests

The following cells test the bidirectional communication between Python and JavaScript.

In [ ]:
# Cell 16: Test Bidirectional Data Transfer - Setup Event Listener
# Set up listener via iife to respond to events
iife("""
    // Remove any existing listener first
    if (window._browserInfoHandler) {
        document.body.removeEventListener('get-browser-info', window._browserInfoHandler);
    }
    
    window._browserInfoHandler = (e) => {
        const data = {
            url: window.location.href,
            title: document.title,
            timestamp: new Date().toISOString(),
            viewport: {
                width: window.innerWidth,
                height: window.innerHeight
            }
        };
        // Push data back to Python
        fetch('/push_data_blocking_', {
            method: 'POST',
            headers: {'Content-Type': 'application/x-www-form-urlencoded'},
            body: `dlg_name=${window.NOTEBOOK_ID}&data_id=${e.detail.idx}&data=${JSON.stringify(data)}`
        });
    };
    document.body.addEventListener('get-browser-info', window._browserInfoHandler);
    console.log('Event listener registered for get-browser-info');
""")
print("Event listener set up!")

In [ ]:
# Cell 17: Trigger Event and Get Response
# This fires an event and waits for the JS listener to respond
try:
    result = event_get('get-browser-info', timeout=5)
    print("Received from browser:")
    print(f"  URL: {result.url}")
    print(f"  Title: {result.title}")
    print(f"  Viewport: {result.viewport.width}x{result.viewport.height}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure you ran the event setup cell first!")

## Cell Execution Tests

The following cells test programmatic cell execution via `run_msg()`.

In [ ]:
# Cell 18: Test run_msg - Queue Cell for Execution
import time

# Create a code cell that will produce output
code_cell_id = add_msg(
    "result = 2 + 2\nprint(f'The answer is {result}')\nresult",
    msg_type="code",
    placement="after"
)
print(f"Created code cell: {code_cell_id}")

# Queue the cell for execution
run_msg(code_cell_id)
print("Cell queued for execution...")

# Wait for execution to complete
time.sleep(2)

# Read the cell to see its output (access via .msg)
result = read_msg(msgid=code_cell_id)
print(f"\nCell output after execution:")
print(f"  Content: {result.msg.content}")
print(f"  Output: {result.msg.output if hasattr(result.msg, 'output') else 'N/A'}")

## Summary

If all cells ran successfully, you've verified:

### Basic Operations
- **curr_dialog()** - Get dialog/notebook info (returns plain dict)
- **read_msg()** - Read cell content (returns AttrDict with .id, .source, etc.)
- **msg_idx()** - Get cell index by ID
- **find_msgs()** - Search cells by type, pattern, etc. (returns list of AttrDicts)

### Cell Modification
- **add_msg()** - Create new cells (returns cell ID string)
- **del_msg()** - Delete cells
- **update_msg()** - Update cell properties (pinned, skipped, etc.)
- **msg_str_replace()** - Replace string in cell
- **msg_insert_line()** - Insert line in cell

### JavaScript Integration
- **iife()** - Execute JavaScript in browser
- **add_html()** - Inject HTML via HTMX OOB

### Bidirectional Communication
- **fire_event()** - Fire browser events from Python
- **event_get()** - Request/response pattern with browser

### Execution
- **run_msg()** - Queue cells for execution

This enables the "Emacs-like" extensibility:
- Modify the UI from Python
- Interact with browser APIs
- Connect to Chrome extensions
- Build custom workflows
- Programmatically create and run code cells